# SQL from Data Analysis Fundamentals

Units 7 & 8

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env file
mysql_password = os.getenv("MYSQL_PASSWORD")

In [ ]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
    database="Test1"
)
cursor = connection.cursor() # 'cursor' is a conventional name for this object that acts as a pointer for sending queries and receiving results

In [ ]:
cursor.execute("SELECT * FROM Person;") # .execute() runs a query
rows = cursor.fetchall() # fetchall returns all results from the previous query

In [ ]:
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
df

## Create and populate a new DB

In [ ]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password
)
cursor = connection.cursor()

In [ ]:
# Create a 'People' database if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS People;")

# Switch to the new database
cursor.execute("USE People;")

In [ ]:
# Create 'Friends' table with the requested attributes
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Friends (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(50),
        gender VARCHAR(10),
        age INT,
        location VARCHAR(50)
        );
        """)


In [ ]:
# Data to insert
friends_data = [
    ("Pablo", "Male", 55, "London"),
    ("Sophie", "Female", 56, "Liphook"),
    ("Sabina", "Female", 44, "London"),
    ("Bubbly", "Female", 54, "London")
]

# Insert data
cursor.executemany("""
   INSERT INTO Friends (name, gender, age, location)
   VALUES (%s, %s, %s, %s);
   """, friends_data)

# Commit the transaction
connection.commit()


In [ ]:
# Fetch all rows to check
cursor.execute("SELECT * FROM Friends;")
rows = cursor.fetchall()

# Show in a nice table using pandas
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
df


## Create a database as directed in BPP materials - Topic 8 - SQL 2

In [ ]:
cursor.execute("CREATE DATABASE IF NOT EXISTS WorldSales;")

# Switch to the new database
cursor.execute("USE WorldSales;")

In [ ]:
cursor.execute("DROP TABLE IF EXISTS salesgroupA;")
cursor.execute("CREATE TABLE salesgroupA (id INT AUTO_INCREMENT PRIMARY KEY, region VARCHAR(20), number_sales INT);")
cursor.execute('INSERT INTO salesgroupA (region, number_sales) VALUES ("Americas", 100);')
cursor.execute('INSERT INTO salesgroupA (region, number_sales) VALUES ("EMEA", 300);')
cursor.execute('INSERT INTO salesgroupA (region, number_sales) VALUES ("Europe", 100);')
cursor.execute('INSERT INTO salesgroupA (region, number_sales) VALUES ("Pacific", 300);')
connection.commit()

In [ ]:
cursor.execute("DROP TABLE IF EXISTS salesgroupB;")
cursor.execute("CREATE TABLE salesgroupB (id INT AUTO_INCREMENT PRIMARY KEY, region VARCHAR(20), number_sales INT);")
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("Americas", 100);')
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("Asia", 500);')
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("Pacific", 300);')
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("EMEA", 200);')
connection.commit()

In [ ]:
# First table
cursor.execute("SELECT * FROM salesgroupA;")
salesA = cursor.fetchall()
print("salesgroupA:")
for row in salesA:
    print(row)

# Second table
cursor.execute("SELECT * FROM salesgroupB;")
salesB = cursor.fetchall()
print("salesgroupB:")
for row in salesB:
    print(row)



In [ ]:
# List all the regions from either or both tables excluding duplicates
union_query = "SELECT region FROM salesgroupA UNION SELECT region FROM salesgroupB;"
# List all the regions from both tables including duplicates
union_all_query = "SELECT region FROM salesgroupA UNION ALL SELECT region FROM salesgroupB;"
# List all the regions that only show up in both tables
intersect_query = "SELECT region FROM salesgroupA INTERSECT SELECT region FROM salesgroupB;"

In [ ]:
# ALL regions that appear either table - UNIQUE value only
cursor.execute(union_query)
all_regions = cursor.fetchall()
print(all_regions)


In [ ]:
# ALL regions that appear either table - allowing duplicates
cursor.execute(union_all_query)
all_regions_with_duplicates = cursor.fetchall()
print(all_regions_with_duplicates)


In [ ]:
# only regions that appear in BOTH tables
cursor.execute(intersect_query)
common_regions = cursor.fetchall()
print(common_regions)


## FROM

In [ ]:
cursor.execute('SELECT * FROM salesgroupA;') # Gets all FROM salesgroupA

## WHERE

In [ ]:
cursor.execute('SELECT * FROM salesgroupA WHERE region = "Pacific";') # Gets all from salesgroupA WHERE region is "Pacific"
pacific_salesA = cursor.fetchall()
print(pacific_salesA)

cursor.execute('SELECT * FROM salesgroupA WHERE number_sales > 100;') # Gets all from salesgroupA WHERE sales is more than 100
sales_over_100 = cursor.fetchall()
print(sales_over_100)

## AS

AS adds or renames a column

In [ ]:
cursor.execute('SELECT region, number_sales AS sales FROM salesgroupA WHERE number_sales > 100;') # Gets all from salesgroupA WHERE sales is more than 100
sales_over_100 = cursor.fetchall()
df = pd.DataFrame(sales_over_100, columns=[desc[0] for desc in cursor.description])
df

## INTO

Use case 1 - Creating a new table from an existing one - DOES NOT WORK IN MySQL

In [36]:
# cursor.execute("DROP TABLE IF EXISTS salesgroupAv2;")
# cursor.execute('SELECT region AS area, number_sales AS sales INTO salesgroupAv2 FROM salesgroupA;')
# ERROR IN MySQL

In [37]:
cursor.execute("DROP TABLE IF EXISTS salesgroupAv2;")
cursor.execute('CREATE TABLE salesgroupAv2 AS SELECT region AS area, number_sales AS sales FROM salesgroupA') # Create new table
connection.commit()
cursor.execute('SELECT * FROM salesgroupAv2;')
new_table = cursor.fetchall()
df = pd.DataFrame(new_table, columns=[desc[0] for desc in cursor.description])
df

,area,sales
0,Americas,100
1,EMEA,300
2,Europe,100
3,Pacific,300


Use case 2 - Save a SINGLE row result into a SQL variable

In [44]:
cursor.execute('SELECT region, number_sales INTO @area, @sales FROM salesgroupA WHERE region = "Europe";')
cursor.execute('SELECT @area;')
area = cursor.fetchone() # fetchall() would also work here but would produce a list with this single result
print(area)

cursor.execute('SELECT @area, @sales;')
result = cursor.fetchone()
# Unpack into Python variables
area, sales = result
print(f"area: {area} and sales: {sales}")

('Europe',)
area: Europe and sales: 100


In [43]:
cursor.execute('SELECT COUNT(*) INTO @total_sales FROM salesgroupA;')
cursor.execute('SELECT @total_sales;')
total_sales = cursor.fetchone()

print(total_sales)

(4,)


## ORDER BY

In [45]:
# Execute query with ORDER BY
cursor.execute("SELECT id, region, number_sales FROM salesgroupA ORDER BY number_sales ASC;")

# Fetch all results
rows = cursor.fetchall()

# Convert to DataFrame
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

df

,id,region,number_sales
0,1,Americas,100
1,3,Europe,100
2,2,EMEA,300
3,4,Pacific,300


## GROUP BY

In [46]:
cursor.execute("""
               SELECT region, SUM(number_sales) AS total_sales
               FROM salesgroupA
               GROUP BY region;
               """)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,region,total_sales
0,Americas,100
1,EMEA,300
2,Europe,100
3,Pacific,300


In [47]:
# The previous example was weak as there were no regions with more than one entry - here I add extra entries for Asia into salesgroupB.
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("Asia", 200);')
cursor.execute('INSERT INTO salesgroupB (region, number_sales) VALUES ("Asia", 450);')
cursor.execute("""
               SELECT region, SUM(number_sales) AS total_sales
               FROM salesgroupB
               GROUP BY region;
               """)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,region,total_sales
0,Americas,100
1,Asia,1150
2,Pacific,300
3,EMEA,200


## INSERT

In [50]:
cursor.execute('INSERT INTO salesgroupA (region, number_sales) VALUES ("North Africa", 200);')
connection.commit()

and with a different order

In [51]:
cursor.execute('INSERT INTO salesgroupA (number_sales, region) VALUES (100, "West Africa");')
connection.commit()

## UPDATE

In [52]:
cursor.execute('UPDATE salesgroupA SET region = "W Africa" WHERE region = "West Africa"');
connection.commit()

## WHERE... NOT EQUAL TO

In [54]:
cursor.execute('SELECT * FROM salesgroupA WHERE region <> "Pacific";') # Gets all from salesgroupA WHERE region is NOT "Pacific"
non_pacific_salesA = cursor.fetchall()
print(non_pacific_salesA)

[(1, 'Americas', 100), (2, 'EMEA', 300), (3, 'Europe', 100), (5, 'North Africa', 200), (6, 'W Africa', 100), (7, 'North Africa', 200), (8, 'W Africa', 100)]
